# SpikeInterface v0.101.2 - Adapted by Rodrigo Noseda - November 2024

SpikeInterface to analyze a multichannel dataset from Cambridge Neurotech Probes. 
The dataset is extracted using open-ephys DAQ and Bonsai-rx (in .bin).
Event_timestamps need some work.

# 0. Preparation <a class="anchor" id="preparation"></a>

In [ ]:
import spikeinterface.full as si
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import os
import csv
import glob
from datetime import datetime
import warnings
warnings.simplefilter("ignore")
%matplotlib widget
print(f"SpikeInterface Version: {si.__version__}")

# 1. Load Recording and Paths <a class="anchor" id="loading"></a>

In [ ]:
# Setting file paths and basic parameters
base_folder = Path('D:/Ephys_C2DRG/')
data_folder = Path("D:/Ephys_C2DRG/2023_9_19/")

recording_paths_list = []
for filename in os.listdir(data_folder):
    if filename.startswith('RawEphysData') and filename.endswith('.bin'):
        recording_paths_list.append(data_folder / filename)
print('Recording Files List:')
print(recording_paths_list)

# parameters associated to the recording in bin format
num_channels = 64 #must know apriori; modify in probe below accordingly.
fs = 30000
gain_to_uV = 0.195
offset_to_uV = 0
rec_dtype = "float32"
time_axis = 0     
time_format = "%H:%M:%S.%f"
n_cpus = os.cpu_count()
n_jobs = n_cpus - 2 #n_jobs = -1 :equal to the number of cores.
job_kwargs = dict(n_jobs=n_jobs, chunk_duration="1s", progress_bar=True)

# 2. Concatenate Recording, Filtering, CMR and Whiten

In [ ]:
#Extract and append recording segments to Baserecording object
recordings_list = []
rec = si.read_binary(recording_paths_list, num_chan=num_channels,sampling_frequency=fs,
                           dtype=rec_dtype, gain_to_uV=gain_to_uV, offset_to_uV=offset_to_uV, 
                           time_axis=time_axis, is_filtered=False)
recordings_list.append(rec)#Appends all extracted rec to a list. Kilosort does not support segments. Use concatenation or sort segments separatedly.
#recording = si.append_recordings(recordings_list)#Creates Object AppendSegmentRecording
recording = si.concatenate_recordings(recordings_list)#Creates Object ConcatenateSegmentRecording
#Filtering recording
recording_f = si.bandpass_filter(recording, freq_min=400, freq_max=5000)
recording_f_cmr = si.common_reference(recording_f, reference='global', operator='median')
recording_f_cmr_w = si.whiten(recording_f_cmr, int_scale=200)
#recording_f_cmr_w

In [ ]:
#Checktraces before saving preprocessed recording (f, cmr, w, clean) to binary
recording_f_cmr_w_seg1 = si.select_segment_recording(recording_f_cmr_w, segment_indices=0)
recording_f_cmr_w_seg2 = si.select_segment_recording(recording_f_cmr_w, segment_indices=1)

recording_layers = dict(seg1=recording_f_cmr_w_seg1, seg2=recording_f_cmr_w_seg2)

w = si.plot_traces(recording_layers, time_range=[300, 500], channel_ids=[8], return_scaled=True, show_channel_ids=True, backend="ipywidgets")

In [ ]:
a = recording_f_cmr_w.get_times(segment_index=0)
b = recording_f_cmr_w_seg1.get_times()
c = recording_f_cmr_w_seg2.get_times()
print(a, b, c)

# 3. Artifact Removal
### Use Silence Periods Function

In [ ]:
# Plot a single channel trace to store the x coordinates on key_press (for remove artifact function)
coordinates_x_end = [] #holds the end timestamp of the short artifact (< 10ms)
coordinates_x_long = [] #holds the start and end timestamp of a long artifact (> 10ms). Usually ~1 sec
start_time = 568 * fs
end_time = 571 * fs
fig, ax = plt.subplots()
trace = recording_f_cmr_w.get_traces(start_frame=start_time, end_frame=end_time, channel_ids=[42], return_scaled=True)
time_axis = np.arange(start_time, end_time) / fs
ax.plot(time_axis, trace)

# Initialize a variable to keep track of the column position (0 or 1)
click_count = 0
# Function to capture click events
def onkey(event):
    global click_count
    if event.key == 'z': # Only respond to the "z" key
        if event.xdata is not None:
            coordinates_x_end.append((event.xdata))# Store the key_press coordinates in the list
            print(f"Key 'z' pressed at: x={event.xdata}")# Display the coordinates
                
    elif event.key == 'w': # Only respond to the "a" key
        if event.xdata is not None:
            if click_count % 2 == 0:
                # Start a new row for each pair of clicks
                coordinates_x_long.append([event.xdata, None])  # Initialize the second column as None
            else:
                # Update the second column for the latest row
                coordinates_x_long[-1][1] = event.xdata
            print(f"Key 'w' pressed at: x={event.xdata}")
            click_count += 1

# Connect the button press event to the figure
cid = fig.canvas.mpl_connect('key_press_event', onkey)
plt.show()

In [ ]:
# Create arrays with times in seconds for artifact removal. Remove 10 ms for TTL artifacts. 
filename = 'artifacts_coordinates_x.csv'
sec = 0.01 # 10ms
arr = np.array([coordinates_x_end][0])
new_col = arr - sec
coordinates_end = np.column_stack((new_col, arr))
coordinates_long = np.array([coordinates_x_long][0])
artifacts_coordinates_x = np.row_stack((coordinates_end, coordinates_long))
# Create new csv file with values (6 decimal points), and append new values to existing csv file
with open(data_folder / filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in artifacts_coordinates_x:
                writer.writerow([f"{value:.6f}" for value in row])

In [ ]:
# Split artifact times per segment for artifact removal and sorting
filename = 'artifacts_coordinates_x.csv'
triggers_in_frames = []
df = pd.read_csv(data_folder / filename, header=None)
df = df.sort_values(by=0).reset_index(drop=True)
#artifact_times = pd.DataFrame({'Start_artifact': df.iloc[:, 0], 'End_artifact': df.iloc[:, 1]})
split_index1 = 461
#split_index2 = 461 #use if more than 2 segments
df2 = df.iloc[:split_index1]
df3 = df.iloc[split_index1:]
# Save each segment to separate files
df2.to_csv(data_folder / "artifacts_segment_1.csv", index=False, header=None)
df3.to_csv(data_folder / "artifacts_segment_2.csv", index=False, header=None)

#Load the two CSV files created earlier
#segment_1 = pd.read_csv("segment_1.csv")
#segment_2 = pd.read_csv("segment_2.csv")

# Create a list containing the two DataFrames
#segments_list = [segment_1, segment_2]

# Print or return the list of segments
#print(segments_list)

In [ ]:
# Create list of tuples of triggers in frames from artifacts_coordinates_x csv file
filename1 = 'artifacts_segment_1.csv'
filename2 = 'artifacts_segment_2.csv'
triggers_in_frames_seg1 = []
triggers_in_frames_seg2 = []
with open(data_folder / filename1, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
         triggers_in_frames_seg1.append([((float(row[0])*fs)), ((float(row[1])*fs))])
with open(data_folder / filename2, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
         triggers_in_frames_seg2.append([((float(row[0])*fs)), ((float(row[1])*fs))])
triggers_in_frames = [triggers_in_frames_seg1, triggers_in_frames_seg2]
print(triggers_in_frames)

In [ ]:
# Create list of tuples of triggers in frames from artifacts_coordinates_x csv file
filename = 'artifacts_coordinates_x.csv'
triggers_in_frames = []
with open(data_folder / filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
         triggers_in_frames.append([((float(row[0])*fs)), ((float(row[1])*fs))])
triggers_in_frames.sort()
print(triggers_in_frames)

In [ ]:
#Clean recording with silence_periods. List periods is one list per segment of tuples (start_frame, end_frame).
recording_f_cmr_w_clean = si.silence_periods(recording_f_cmr_w, list_periods=triggers_in_frames, seed=0, mode='zeros')

#recording_layers = dict(filtered=recording_f_cmr, whitened=recording_f_cmr_w, w_cleaned=recording_f_cmr_w_clean,)

In [ ]:
#Checktraces before saving preprocessed recording (f, cmr, w, clean) to binary
w0 = si.plot_traces(recording_f_cmr_w, time_range=[0, 1], channel_ids=[8], segment_index=0,
                   return_scaled=True, show_channel_ids=True, backend="ipywidgets")

# 6. Saving filtered, clean, motion corrected and whitened recording (with probe) to binary.

In [ ]:
recording_f_cmr_w_clean.save(format='binary', folder=data_folder / "recording_preprocessed_200", overwrite=True, **job_kwargs)